In [ ]:
#| default_exp ai_ass_label

In [ ]:
#| export
import os
import pandas as pd
import openai
import random
import matplotlib.pyplot as plt
import os
import re

### Apply color based on confidence score

In [ ]:
#| export 
def apply_color(val):
    # Set color corresponding to the confidence score
    if float(val) > 0.7:
        color = 'green'
    elif float(val) < 0.3:
        color = 'red'
    else:
        color = "yellow"

    return f'background-color: {color}'

### Classified unlabeled texts one by one

In [ ]:
#| export
def classify_unlabeled_texts(df, text_column, label_column):
    # Filter to get labeled examples for the prompt
    labeled_examples = df[df[label_column].notna()]
    openai.api_key = os.getenv('OPENAI_API_KEY')

    # Iterate over rows that need classification
    for index, row in df[df[label_column].isna()].iterrows():
        messages = [{"role": "system", "content": "The following are examples of texts and their classifications, the label is either 'OTR', 'PRS', 'REP', or 'NEU'. 'OTR' stands for 'An opportunity to respond' (e.g., 'What color is this candy?'), 'PRS' stands for 'Praise' (e.g., 'Great job, Morgan!'), 'REP' stands for 'Reprimand' (e.g., 'You need to keep quiet while someone else is reading.) and 'NEU' stands for 'None of the above':"}]
        
        # Add few-shot learning examples
        for _, example_row in labeled_examples.iterrows():
            example = f"Text: {example_row[text_column]}\nLabel: {example_row[label_column]}"
            messages.append({"role": "user", "content": example})

        # Add the text to be classified
        text_to_classify = row[text_column]
        messages.append({"role": "user", "content": f"Text: {text_to_classify}\nLabel:"})

        # Make the request to OpenAI's chat API
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=messages
        )

        # Update the DataFrame with the classified label
        classified_label = response.choices[0].message['content'].strip()
        df.at[index, label_column] = classified_label

    return df

### Classified unlabeled texts at once

In [ ]:
#| export
def classify_unlabeled_texts_all(df, text_column, label_column):
    # Filter to get labeled examples for the prompt
    labeled_examples = df[df[label_column].notna()]
    openai.api_key = os.getenv('OPENAI_API_KEY')

    prompt =  "You are a Label assistant that could help label unlabeled texts into labels, the following are examples of texts and their corresponding labels, the label is either 'OTR', 'PRS', 'REP', or 'NEU'. 'OTR' stands for 'An opportunity to respond' (e.g., 'What color is this candy?'), 'PRS' stands for 'Praise' (e.g., 'Great job, Morgan!'), 'REP' stands for 'Reprimand' (e.g., 'You need to keep quiet while someone else is reading.) and 'NEU' stands for 'None of the above':"
    
    few_shot_examples_texts = ""
    for _, example_row in labeled_examples.iterrows():
        few_shot_examples_texts += f"Texts:{example_row[text_column]}\nLabels: {example_row[label_column]}\n"

    prompt += few_shot_examples_texts
    prompt += "Now Label the following texts:\n"

    text_to_classify = ""

    # Iterate over rows that need classification
    for index, row in df[df[label_column].isna()].iterrows():
        # Add the text to be classified
        text_to_classify += f"Texts: {row[text_column]}\n"

    prompt += text_to_classify

        # Make the request to OpenAI's chat API
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": str(prompt)}]
    )
    
    contents = response.choices[0].message['content'].strip()
    # Extract all the labels
    # use regex to extract the labels 'OTR', 'PRS', 'REP', or 'NEU' from the response
    regex = r"(OTR|PRS|REP|NEU)"
    classified_labels = re.findall(regex, contents)
    # Update the DataFrame with the classified label
    
    for index, row in df[df[label_column].isna()].iterrows():
        try:
            df.at[index, label_column] = classified_labels.pop(0)
        except:
            df.at[index, label_column] = None
    return df

### AI Assisted labeling (simple)

In [ ]:
#| export
def ai_assisted_labeling(uploaded_file, label_column = 'Label', text_column='Text'):
    # Placeholder for AI Labeling Logic
    df = pd.read_csv(uploaded_file.name, encoding='utf-8')

    # if label_column not in df.columns, create a new column with the label_column name
    if label_column not in df.columns:
        df[label_column] = None
    df = df[[text_column, label_column]]
    # Process the dataframe
    # prompt = generate_prompt(df)

    # response = inference_by_openai(prompt)

    # add random confidence score to each row in confidence column, the confidence score is between 0 and 1
    df['Confidence'] = [round(random.random(), 2) for _ in range(len(df))]
    
    # set confidence as 1 is the label is in a valid list of labels
    df.loc[df['Label'].isin(['PRS', 'OTR', 'REP', 'NEU']), 'Confidence'] = 1
    
    # def apply_color(var):
    #     color = 'red' if var is None else 'green'
    #     return f'background-color: {color}'

    style_df = df.style.applymap(apply_color, subset=['Confidence'])

    df = classify_unlabeled_texts_all(df, 'Text', 'Label')

    # Save the new dataframe to a CSV file
    new_filename = uploaded_file.name.split('.')[0] + '_AI_assisted.csv'
    df.to_csv(new_filename, index=False)
    

    return style_df.to_html(), new_filename

### AI Assisted Labeling (Advanced)

In [ ]:
#| export
def start_to_label(uploaded_file, chunk_size, accuracy_log, chunk_count, text_column="Text", label_column="Label"):
    # initial the accuracy log and chunk count when it is started
    accuracy_log = []
    chunk_count = 0
    df = pd.read_csv(uploaded_file.name, encoding='utf-8')
    if label_column not in df.columns:
        df[label_column] = None
    df = df[[text_column, label_column]]
    return df.head(int(chunk_size)), df, accuracy_log, chunk_count

In [ ]:
#| export
def correct_dataframe(dataframe_display, file_input, current_dataframe, accuracy_log, chunk_size, chunk_count, text_column = "Text", label_column="Label"):
    max_chunk_count = len(current_dataframe) / chunk_size if len(current_dataframe) % chunk_size == 0 else int(len(current_dataframe) / chunk_size) + 1
    
    temp_df = current_dataframe.copy()

    # only iterate through the rows that are not empty
    
    for _, row in dataframe_display[~dataframe_display[label_column].isna()].iterrows():
        # current_dataframe[current_dataframe[text_column] == row[text_column]][label_column] = row[label_column]

        # use .loc[row_indexer,col_indexer] = value instead
        current_dataframe.loc[current_dataframe[text_column] == row[text_column], label_column] = row[label_column]

    # Check if "_AI_assisted.csv" is in the name of file_output
    accuracy = None

    if chunk_count > 0:
        compared_chunk_temp = temp_df.loc[min(chunk_size * (chunk_count - 1), len(temp_df) - chunk_size): min(chunk_size * chunk_count - 1, len(temp_df) - 1)]
        compared_chunk_current = current_dataframe.loc[min(chunk_size * (chunk_count - 1), len(current_dataframe) - chunk_size): min(chunk_size * chunk_count - 1, len(current_dataframe) - 1)]
        # Calculate the accuracy based on how many rows of the dataframe_display is different from the current_dataframe
        merged_df = pd.merge(compared_chunk_temp, compared_chunk_current, on=text_column, suffixes=('_before', '_current'))
        merged_df.dropna(subset=[f'{label_column}_current', f'{label_column}_before'], inplace=True)

        # with open('merged_df.csv', 'w') as f:
        #     f.write(merged_df.to_csv(merged_df[f'{label_column}_before'] != merged_df[f'{label_column}_current']).sum())
        accuracy = 1 - (merged_df[f'{label_column}_before'] != merged_df[f'{label_column}_current']).sum() / len(merged_df)
        if len(accuracy_log) < max_chunk_count - 1:
            accuracy_log.append(accuracy)

    if accuracy != 1:
        current_dataframe.loc[min(chunk_size * chunk_count, len(current_dataframe) - chunk_size): , label_column] = None
        current_dataframe = classify_unlabeled_texts_all(current_dataframe, 'Text', 'Label')

    new_file = file_input.name.split('.')[0] + '_AI_assisted.csv'

    current_dataframe.to_csv(new_file)

    dataframe_display = current_dataframe.loc[min(chunk_size * chunk_count, len(current_dataframe) - chunk_size): min(chunk_size*(chunk_count+1)-1, len(current_dataframe) - 1)]

    if chunk_count < max_chunk_count - 1:
        chunk_count += 1

    return dataframe_display, new_file, current_dataframe, accuracy_log, chunk_count

In [ ]:
#| export
def create_plot(accuracy_log, save_path="plot.png"):
    ax = plt.figure()
    # X is the index of the accuracy_log, y is the value in accuray_log
    plt.plot(range(len(accuracy_log)), accuracy_log)
    plt.xlabel('Batch Index')
    plt.ylabel('Batch Accuracy')
    plt.title('Batch Accuracy over time')

    # Save the plot to a BytesIO object
    # buf = io.BytesIO()
    plt.savefig(save_path, format='png')
    # buf.seek(0)

    # Return the plot and the file for download
    return ax, save_path